# Day 2

Great, it's another one of these coordinate puzzles.  We're going to get a set of directions that our boat is going to take.  I like these, as I got my programming career started with games programming, and so anything to do with coordinates, maps, and moving things around is a set of problems for which I already have some of the tools.

There's a fun twist on this one.  In previous years, it's often been about rotating, or moving north, east, south and west.  But in this case, we've only got 3 instructions, forward, up and down.  Furthermore, our depth is measured as a distance from the surface of the sea, so going down is going to increase our depth.  i.e. if we're at the surface, we're at depth 0, but going down will take us down to depth 1, and so on.

In this case, we need to know what depth we end up with.
We can do this the simple to understand, but harder to implement way, or we can do it the mathmatical way, that's easier to implement, but a little harder to understand.

Let me explain teh second, and then we'll do it the "proper" way.

We've got 3 commands, and if we were plotting these on a y,z axis, the commands result in a change to the y axis, or to the z axis.  However, in this case, we've only been asked for the endpoint, which means we don't actually care about any intermediate numbers or steps, we just care about the end total.

So we could, for the y axis, simply take all the forwards, add all the numbers together, and that will be the result.
We could also take all the down's add them together, and then subtract the total of all the ups, and that would be our resultant depth.

However, I suspect we'll want to look more closely at this, and possibily at something like the deepest point we've ever got to, so instead we're going to create a submarine data structure, and then parse each instruction and update the y or z position respectively.

In [1]:
## Import ipytest and get it setup for use in Python Notebook
import pytest
import ipytest
ipytest.autoconfig()

This time, we're going to build our parser first, because I want to turn each line into an instruction, so the parser is more complex.

How are we going to do that?

We're going to read the line, and if it's "forward 15", we'll turn it into a "vector".  In this case it'll just be a tuple, although Python has a handy namedtuple class so I'll use that.  We're going to define the y coordinate as starting at 0 and incrementing as the sub goes forward.  The z oordinate will represent the depth from the bottom

In [2]:
test_data_raw = """forward 5
down 5
forward 8
up 3
down 8
forward 2""".split("\n")

from collections import namedtuple
Sub = namedtuple('Sub', ['y', 'z'])
Vector = namedtuple('Vector', ['y', 'z'])

So we've got two named tuples, one for the sub and one for the Vector.  We'll show later how we can add one to another, but first our parser needs to turn a line of text into a Vector

In [3]:
def parse_line(line):
    if line.startswith("forward"):
        y = int(line.replace("forward ",""))
        return Vector(y,0)
    if line.startswith("down"):
        z = int(line.replace("down ",""))
        return Vector(0,z)
    if line.startswith("up"):
        z = -int(line.replace("up ",""))
        return Vector(0,z)
    return None

assert parse_line("forward 5") == Vector(5,0)
assert parse_line("down 5") == Vector(0,5)
assert parse_line("up 3") == Vector(0,-3)

So we can now turn a list of intructions into a list of vectors, let's check that against the test data nad make sure it works.

In [4]:
def parse(lines):
    return [parse_line(line) for line in lines]

test_data = parse(test_data_raw)
assert test_data == [Vector(5,0),Vector(0,5),Vector(8,0),Vector(0,-3),Vector(0,8),Vector(2,0)]

Now that we've got the data into vectors, we can pretty simply move hte submarine by adding the sub and the vector together.  We however need to define to python how that addition works, because by default it will just assume we have tuples, and adding two tuples together is treated like concatonation, rather than fieldwise addition.

We need to redefine the Sub class here, because it's a pain to add a method to an existing class, luckily we hadn't really used Sub anywhere yet.  Strike one for the PythonNotebook though, I need to remember not to define things until I need them.

In [5]:
class Sub(namedtuple('Sub', ['y', 'z'])
):
    def __add__(self, v):
        return Sub(self.y+v.y, self.z+v.z)

assert Sub(3,8) == Sub(1,3)+Vector(2,5)

Now we can walk the list of vectors we got, moving the sub at each point, and we'll get a Sub with end position at the end

In [6]:
def moveSub(sub, directions):
    for movement in directions:
        sub = sub + movement
    return sub

testSub = Sub(0,0)
testSub = moveSub(testSub, test_data)
assert Sub(15,10) == testSub
assert 150 == testSub.y*testSub.z

Let's do it on real data

In [7]:
data = parse([line.strip() for line in open("day2.txt").readlines()])
sub = moveSub(Sub(0,0), data)
assert Sub(1957,955) == sub
assert 1868935 == sub.y*sub.z

Cool, onto 

## Part 2

Oh great, so instead of our parsing meaning that we're changing the vector, what we need to do is something slightly more complex, we need our Sub to have an aim.
This means we're going to need to store 3 values for our Sub, our y, our z and our aim.
When we get a forward command we apply a change to our y and our z, and when we get an up or down command, we can change our aim.

We can probably do this without changing our vector, because our parser turns the up and down commands into a positive or negative z, and we can treat that as just a change to aim.

Let's build a new sub and demonstrate with the first few test data examples

In [8]:
class Sub(namedtuple('Sub', ['y', 'z', 'aim'])
):
    def __add__(self, v):
        if v.y > 0:
            newz = self.z+self.aim*v.y
            return Sub(self.y+v.y, newz, self.aim)
        return Sub(self.y, self.z, self.aim+v.z)

testsub = Sub(0,0,0)
testsub = testsub+Vector(5,0)
assert Sub(5,0,0) == testsub
testsub+=Vector(0,5)
assert Sub(5,0,5) == testsub
testsub+=Vector(8,0)
assert Sub(13,40,5) == testsub
testsub+=Vector(0,-3)
assert Sub(13,40,2) == testsub
testsub+=Vector(0,8)
assert Sub(13,40,10) == testsub
testsub+=Vector(2,0)
assert Sub(15,60,10) == testsub

Great, that was easier than expected.  If I was doing this in normal python, I'd probably rename the z component of a Vector to be called deltaaim, or aimchange, which would make it more readable.

Let's retry running the parsed data on the test data and see what happens

In [9]:
testSub = Sub(0,0,0)
testSub = moveSub(testSub, test_data)
assert Sub(15,60,10) == testSub
assert 900 == testSub.y*testSub.z

Brilliant.  And lets go again on real data

In [10]:
sub = moveSub(Sub(0,0,0), data)
assert Sub(1957,1004584,955) == sub
total = sub.y*sub.z
assert 1965970888 == total